In [1]:
import os
import torch
import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Load metadata CSV file
path = "C:/Users/annammc/Documents/Gestures"
# path = "/itf-fi-ml/shared/users/annammc/gestures/mocap_csv" FOR ML NODE
audiofile_list = glob.glob(os.path.join(path, "*.wav"))
videofile_list = glob.glob(os.path.join(path, "*.mov"))
midifile_list = glob.glob(os.path.join(path, "*.mid"))
mocapcsv_list = glob.glob(os.path.join(path, "*.csv"))


In [4]:
for item in mocapcsv_list:
    df = pd.read_csv(item)
    basename = os.path.splitext(os.path.basename(item))[0]

    LHAND_x = df['LHAND_x'].to_numpy()
    LHAND_y = df['LHAND_y'].to_numpy()
    LHAND_z = df['LHAND_z'].to_numpy()
    RHAND_x = df['RHAND_x'].to_numpy()
    RHAND_y = df['RHAND_y'].to_numpy()
    RHAND_z = df['RHAND_z'].to_numpy()

    # Compute velocity for each hand
    LHAND_velocity_x = np.gradient(LHAND_x)
    LHAND_velocity_y = np.gradient(LHAND_y)
    LHAND_velocity_z = np.gradient(LHAND_z)

    RHAND_velocity_x = np.gradient(RHAND_x)
    RHAND_velocity_y = np.gradient(RHAND_y)
    RHAND_velocity_z = np.gradient(RHAND_z)

    # Compute acceleration for each hand 
    LHAND_acceleration_x = np.gradient(LHAND_velocity_x)
    LHAND_acceleration_y = np.gradient(LHAND_velocity_y)
    LHAND_acceleration_z = np.gradient(LHAND_velocity_z)

    RHAND_acceleration_x = np.gradient(RHAND_velocity_x)
    RHAND_acceleration_y = np.gradient(RHAND_velocity_y)
    RHAND_acceleration_z = np.gradient(RHAND_velocity_z)

    # Stack the velocity and acceleration arrays for each hand
    mocap_feature = np.vstack((LHAND_velocity_x, LHAND_velocity_y, LHAND_velocity_z,
                               LHAND_acceleration_x, LHAND_acceleration_y, LHAND_acceleration_z,
                               RHAND_velocity_x, RHAND_velocity_y, RHAND_velocity_z,
                               RHAND_acceleration_x, RHAND_acceleration_y, RHAND_acceleration_z))


    np.save(f"C:/Users/annammc/Downloads/Anna/MoCap/mocap/{basename}.npy", mocap_feature)
    # np.save(f"/itf-fi-ml/shared/users/annammc/Anna/mocap2/{basename}.npy", mocap_feature) FOR ML NODE

        

In [5]:
db_gesture = "C:/Users/annammc/Downloads/Anna/MoCap/mocap/*"
# db_gesture = "/itf-fi-ml/shared/users/annammc/Anna/mocap2/*" FOR ML NODE
files = glob.glob(db_gesture)
filename = [os.path.basename(item) for item in files]

target = [item.split("_")[-3] for item in files]

label_encoder = LabelEncoder()
target_idx = label_encoder.fit_transform(target) # Change class names to numeric

dict = {'mocap_npy': filename, 'target': target, 'target_idx': target_idx}
dataset_all = pd.DataFrame(dict)

dataset_all.to_csv('metadata.csv')


In [6]:
X = dataset_all['mocap_npy']
y = dataset_all['target_idx']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [7]:
train_df = pd.DataFrame({'filename': list(X_train), 'target': list(y_train)})
test_df = pd.DataFrame({'filename':list(X_test), 'target': list(y_test)})

train_df.to_csv("C:/Users/annammc/Downloads/Anna/MoCap/mocap/train.csv") 
test_df.to_csv("C:/Users/annammc/Downloads/Anna/MoCap/mocap/test.csv") 
# train_df.to_csv("/itf-fi-ml/shared/users/annammc/Anna/mocap2/train.csv") 
# test_df.to_csv("/itf-fi-ml/shared/users/annammc/Anna/mocap2/test.csv")

In [40]:
# X = dataset_all['mocap_npy']
# y = dataset_all['target_idx']

# # Define the number of folds
# num_folds = 5

# # Perform K-Fold cross-validation
# kf = KFold(n_splits=num_folds, shuffle=True)

# fold = 0
# for train_indices, test_indices in kf.split(X):
#     print(f"Fold {fold + 1}/{num_folds}")
    
#     # Split the data for this fold
#     X_train, X_test = X[train_indices], X[test_indices]
#     y_train, y_test = y[train_indices], y[test_indices]
    
#     train_df = pd.DataFrame({'filename': list(X_train), 'target': list(y_train)})
#     test_df = pd.DataFrame({'filename':list(X_test), 'target': list(y_test)})
    
#     train_df.to_csv(f"train_fold{fold+1}.csv")
#     test_df.to_csv(f"test_fold{fold+1}.csv")
    
#     fold += 1

Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5
